## Логистическая регрессия. Анализ номинативных данных  
Это расширение классической линейной регрессии, заточенное под анализ связи независимой переменной  
и бинарной зависимой (переменной с двумя градациями).  
В некотором смысле на логистическую регрессию можно взглянуть как на t-критерий наоборот  
(в t-критерии мы проверяем, как две группы различаются по одной количественной переменной,  
а в логистической регрессии мы проверяем, как одна или несколько количественных переменных  
влияют на возникновение одной или другой группы).

### Задача  
На основе данных о пассажирах корабля титаника оценим шансы пассажира выжить.   

### Модель без предикторов (анализ без участия предикторов)  
Вычислим натуральный логарифм шансов выжить (intercept модели логистической регрессии).

In [1]:
import pandas as pd

titanic_data = pd.read_csv('./data/titanik_full_data.csv', sep='\t')

survived_dead_count  = titanic_data[['PassengerId', 'Survived']].groupby(['Survived']).count()

survived_count = survived_dead_count.loc[1, 'PassengerId']
dead_count = survived_dead_count.loc[0, 'PassengerId']

Вычислим общие шансы выжить при кораблекрушении. 

In [2]:
odds = survived_count / dead_count
print('odds:', odds)

odds: 0.6061349693251534


Вычилим intercept модели логистической регрессии.

In [3]:
import math

intercept = math.log(odds)
print('intercept:', intercept)

intercept: -0.5006525960529401


Проверим гипотезу о том, что распределение частот показателя выживаемости значимо отличается от равномерного.  
H0: intercept = ln(odds) = 0 (odds = 1)

In [4]:
import scipy.stats as st

se = 0.057 # TODO how to calculate standard error?
print('se:', se)

Z = (intercept - 0) / se
print('Z:', Z)

p_value = st.norm.cdf(Z) * 2
print('p_value:', p_value)

alpha = 0.05
if p_value > alpha:
    print(f'p_value > alpha. Do not reject H0')
else:
    print(f'p_value < alpha. Reject H0')

se: 0.057
Z: -8.783378878121756
p_value: 1.5863569516489108e-18
p_value < alpha. Reject H0


In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as sf

sf.glm('Survived ~ 1', titanic_data, family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                 1309
Model:                            GLM   Df Residuals:                     1308
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -867.57
Date:                Fri, 25 Jun 2021   Deviance:                       1735.1
Time:                        15:05:09   Pearson chi2:                 1.31e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5007      0.057     -8.780      0.000      -0.612      -0.389
==============================================================================
"""

### Модель с одним номинативным предиктором  

Анализ с участием одного предиктора - пола.  

Построим таблицу выживаемости пассажиров в зависимости от пола.

In [6]:
titanic_crosstab = pd.crosstab(titanic_data['Survived'], titanic_data['Sex'], margins=False)
print(titanic_crosstab)

Sex       female  male
Survived              
0             81   734
1            385   109


Рассчитаем шансы выжить для мужчин.

In [7]:
odds_man = titanic_crosstab.loc[1, 'male'] / titanic_crosstab.loc[0, 'male']
print('odds_man:', odds_man)
print('ln(odds_man):', math.log(odds_man))

odds_man: 0.14850136239782016
ln(odds_man): -1.907161146385372


Рассчитаем шансы выжить для женщин.

In [8]:
odds_female = titanic_crosstab.loc[1, 'female'] / titanic_crosstab.loc[0, 'female']
print('odds_female:', odds_female)
print('ln(odds_female):', math.log(odds_female))

odds_female: 4.753086419753086
ln(odds_female): 1.5587941796153455


Рассчитаем коэффициент b1 модели, как натуральный логарифм отношения шансов выжить для мужчин и женщин.

In [9]:
odds_ratio = odds_man / odds_female
print('odds_ratio:', odds_ratio)
print('b1: ln(odds_ratio):', math.log(odds_ratio))

odds_ratio: 0.031243143777203723
b1: ln(odds_ratio): -3.465955326000717


In [10]:
sf.glm('Survived ~ C(Sex)', titanic_data, family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                 1309
Model:                            GLM   Df Residuals:                     1307
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -539.84
Date:                Fri, 25 Jun 2021   Deviance:                       1079.7
Time:                        15:05:10   Pearson chi2:                 1.31e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.5588      0.122     12.752      0.000       1.319       1.798
C(Sex)[T.male]    -3.4660      0.160    -21.713      0.000      -3.779      -3.153
==================================================================================
"""

Проведём регрессионный анализ, где зависимая переменная будет выживаемостью пассажира, предиктором - тип билета (три класса).

In [11]:
sf.glm('Survived ~ C(Pclass)', titanic_data, family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                 1309
Model:                            GLM   Df Residuals:                     1306
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -821.91
Date:                Fri, 25 Jun 2021   Deviance:                       1643.8
Time:                        15:05:10   Pearson chi2:                 1.31e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.3058      0.113      2.716      0.007       0.085       0.526
C(Pclass)[T.2]    -0.6188      0.166     -3.733      0.000      -0.944      -0.294
C(Pclass)[T.3]    -1.3035      0.141     -9.254      0.000      -1.580      -1.027
==================================================================================
"""

### Модель с двумя номинативными предикторами  
Проведём регрессионный анализ двух номинативных переменных, где в качестве зависимой переменная будет  
выживаемостью пассажира, предикторами - пол и тип билета.  
3 гипотезы:  
* о значимости фактора "пол"  
* о значимости фактора "класс билета"  
* о взаимодействии факторов "пол" и "класс билета"

In [12]:
glm = sf.glm('Survived ~ C(Sex) * C(Pclass)', titanic_data, family=sm.families.Binomial()).fit()
glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                 1309
Model:                            GLM   Df Residuals:                     1303
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -486.58
Date:                Fri, 25 Jun 2021   Deviance:                       973.16
Time:                        15:05:10   Pearson chi2:                 1.31e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         3.8501      0.583      6.599      0.000       2.707       4.994
C(Sex)[T.male]                   -4.9413      0.608     -8.122      0.000      -6.134      -3.749
C(Pclass)[T.2]                   -1.0367      0.719     -1.442      0.149      -2.446       0.373
C(Pclass)[T.3]                   -3.1570      0.601     -5.252      0.000      -4.335      -1.979
C(Sex)[T.male]:C(Pclass)[T.2]    -0.0758      0.782     -0.097      0.923      -1.609       1.458
C(Sex)[T.male]:C(Pclass)[T.3]     1.9980      0.644      3.104      0.002       0.736       3.260
=================================================================================================
"""

### Интерпретация результатов анализа  
* Intercept - ln(шансов положительного исхода для женщин в первом классе).  
* C(Sex)\[T.male] - ln(отношения шансов выжить для мужчин и женщин в 1-ом классе) или  
по свойству логарифма: ln(шансов выжить для мужчин в 1-ом классе) - ln(шансов выжить для женщин в 1-ом классе).  
Значение b1 = -4.9413 при P>|z| = 0.000 говорит о том, что шансов выжить у мужчин значимо меньше, чем у женщин в 1-ом классе.  
* C(Pclass)\[T.2] - ln(отношения шансов выжить для женщин в 1-ом классе и во 2-ом классе).  
Шансы выжить для женщин 1-го и 2-го класса отличаются статистически не значимо (P>|z| = 0.149 > 0.05).  
* C(Pclass)\[T.3] - ln(отношения шансов выжить для женщин в 1 классе и во 3 классе).  
Шансы выжить для женщин 1-го и 3-го класса статистически значимо отличаются (P>|z| = 0.000).  

Взаимодействие предикторов  
* C(Sex)\[T.male]:C(Pclass)\[T.2] - разность ln(отношения шансов выжить для мужчин и женщин 2-го класса)  
и ln(отношения шансов выжить для мужчин и женщин 1-го класса)  
Значение -0.0758 при P>|z| = 0.923 говорит о том, что изменение отношения шансов выжить для мужчин к шансам выжить для женщин  
для разных классов статистически незначимо.
* C(Sex)\[T.male]:C(Pclass)\[T.3] - разность ln(отношения шансов выжить для мужчин и женщин 3-го класса)  
и ln(отношения шансов выжить для мужчин и женщин 1-го класса)  
Значение 1.9980	 при P>|z| = 0.002 говорит о том, что изменение отношения шансов выжить для мужчин к шансам выжить для женщин  
для разных классов статистически значимо. Существует взаимосвязь между полом и тем, выжил ли человек или не выжил, для 1-го и 3-го класса.


### Прогнозирование шансов

In [13]:
ln_odds_female_class1 = glm.predict({'Sex': ['female'], 'Pclass': [1]}, linear=True)
print('Intercept = ln_odds_female_class1:', ln_odds_female_class1[0])

Intercept = ln_odds_female_class1: 3.8501476017100518


Рассчитаем шансы выжить для женщин в 1 классе.

In [14]:
odds_female_class1 = math.exp(ln_odds_female_class1)
print('odds_female_class1:', odds_female_class1)

odds_female_class1: 46.99999999999968


Рассчитаем вероятность выжить для женщин в 1 классе.

In [15]:
p_female_class1 = odds_female_class1 / (1 + odds_female_class1)
print('p_female_class1:', p_female_class1)

p_female_class1: 0.9791666666666665


In [16]:
p_female_class1 = glm.predict({'Sex': ['female'], 'Pclass': [1]})
print('p_female_class1:', p_female_class1[0])

p_female_class1: 0.9791666666666665


## Логистическая регрессия. Количественные и номинативные данные

На основе данных о пассажирах корабля титаника оценим шансы пассажира выжить,  
учитывая номинативные переменные "пол" и "класс", а также количественную - "возраст"

In [17]:
glm = sf.glm('Survived ~ C(Sex) + C(Pclass) + Age', titanic_data, family=sm.families.Binomial()).fit()
glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                 1046
Model:                            GLM   Df Residuals:                     1041
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -398.21
Date:                Fri, 25 Jun 2021   Deviance:                       796.42
Time:                        15:05:10   Pearson chi2:                 1.10e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.9568      0.372     10.641      0.000       3.228       4.686
C(Sex)[T.male]    -3.5601      0.196    -18.158      0.000      -3.944      -3.176
C(Pclass)[T.2]    -1.2370      0.256     -4.833      0.000      -1.739      -0.735
C(Pclass)[T.3]    -2.2390      0.256     -8.759      0.000      -2.740      -1.738
Age               -0.0313      0.007     -4.407      0.000      -0.045      -0.017
==================================================================================
"""

Intercept в данном случае равен ln(шансов положительного исхода для женщин в первом классе при возрасте = 0).  

Рассчитаем, на сколько будет влиять увеличение возраста на 1 год, для женщины в 1 классе, на вероятность выживания.

In [18]:
ln_odds_female_class1_0_year = glm.predict({'Sex': ['female'], 'Pclass': [1], 'Age': [0]})
ln_odds_female_class1_1_year = glm.predict({'Sex': ['female'], 'Pclass': [1], 'Age': [1]})
print(math.exp(ln_odds_female_class1_1_year[0])-math.exp(ln_odds_female_class1_0_year))

-0.0015597927703976389
